# BRIEF
IT NOW CAN PERFORM REGRESSION AND multi class CLASSIFICATION



In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape and add channel dimension
train_images = train_images.reshape((60000, 28, 28, 1))  # [B, H, W, C]
test_images = test_images.reshape((10000, 28, 28, 1))    # [B, H, W, C]

# Convert to [B, C, H, W] format  # [B, C, H, W]

# Normalize pixel values to the range [0, 1]
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

train_images = train_images.transpose(0, 3, 1, 2)
test_images = test_images.transpose(0, 3, 1, 2)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Print shapes
print("Train images shape:", train_images.shape)  # Should be (60000, 1, 28, 28)
print("Train labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)    # Should be (10000, 1, 28, 28)
print("Test labels shape:", test_labels.shape)

Train images shape: (60000, 1, 28, 28)
Train labels shape: (60000, 10)
Test images shape: (10000, 1, 28, 28)
Test labels shape: (10000, 10)


In [ ]:
print(type(train_labels))

In [ ]:
import numpy as np
from core.Models import Model
from core.Function import Tanh, Softmax, Relu
from core.nn import Linear, Conv2d, MaxPool2d
from core.optim import sgd, adam
from core.loss import get_loss_fn

class ResNetWithResiduals(Model):
    def __init__(self):
        super().__init__()

        # Initial Convolution
        self.conv1 = Conv2d(input_channels=1, output_channels=8, kernel_size=3, stride=1, padding=1, initialize_type='xavier')
        self.relu1 = Relu()

        # Residual Block 1
        self.conv2 = Conv2d(input_channels=8, output_channels=16, kernel_size=3, stride=1, padding=1, initialize_type='xavier')
        self.relu2 = Relu()
        self.res1 = Conv2d(input_channels=8, output_channels=16, kernel_size=1, stride=1, padding=0, initialize_type='xavier')  # Adjust channels
        self.max1 = MaxPool2d(kernel_size=2, stride=2)

        # Residual Block 2
        self.conv3 = Conv2d(input_channels=16, output_channels=32, kernel_size=3, stride=1, padding=1, initialize_type='xavier')
        self.relu3 = Relu()

        self.conv4 = Conv2d(input_channels=32, output_channels=64, kernel_size=3, stride=1, padding=1, initialize_type='xavier')
        self.relu4 = Relu()
        self.res2 = Conv2d(input_channels=16, output_channels=64, kernel_size=1, stride=1, padding=0, initialize_type='xavier')  # Adjust channels
        self.max2 = MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer
        self.linear1 = Linear(64 * 7 * 7, 100, initialize_type='xavier', activation='tanh')
        self.linear2 = Linear(100, 10, initialize_type='xavier')
        self.softmax = Softmax()

    def forward(self, x):
        # Initial conv
        x1 = self.conv1(x)
        x1 = self.relu1(x1)

        # Residual Block 1
        x2 = self.conv2(x1)
        x2 = self.relu2(x2)
        res1 = self.res1(x1)  # Match channel dimensions
        x2 += res1  # Add residual connection
        x2 = self.max1(x2)

        # Residual Block 2
        x3 = self.conv3(x2)
        x3 = self.relu3(x3)

        x4 = self.conv4(x3)
        x4 = self.relu4(x4)
        res2 = self.res2(x2)  # Match channel dimensions
        x4 += res2  # Add residual connection
        x4 = self.max2(x4)

        # Fully Connected Layer
        x4 = self.linear1(x4)
        x4 = self.linear2(x4)
        x4 = self.softmax(x4)

        return x4

# Instantiate model
model_resnet_residuals = ResNetWithResiduals()


In [2]:
from core.Datasets import Dataset
train_dataset = Dataset(train_images, train_labels, batch_size=32, shuffle=True)

In [ ]:
optimizer = adam(model_resnet_residuals.parameters(), learning_rate=0.001)
loss_fn = get_loss_fn('categorical_bce')
num_epochs = 2
def train():
    for epoch in range(num_epochs):
        train_dataset.reset()  # Reset dataset iterator and reshuffle if needed
        epoch_loss = 0.0
        num_batches = 0
        for X_batch, y_batch in train_dataset:
            optimizer.zero_grad()
            out = model_resnet_residuals(X_batch)
            loss_tensor = loss_fn.sparse_categorical_cross_entropy(y_batch, out)
            epoch_loss += loss_tensor.data
            num_batches += 1
            loss_tensor.backward()
            optimizer.step()
        avg_loss = epoch_loss / num_batches
        
        print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}")




In [ ]:
train()

In [ ]:
def test():
    x = model_resnet_residuals(test_images)
    y_pred = x.data
    y_pred_classes = np.argmax(y_pred, axis=1, keepdims=True)
    y_true_classes = np.argmax(test_labels, axis=1, keepdims=True)  # Assuming test_labels are integers
    accuracy = np.mean(y_pred_classes == y_true_classes)
    print(f"Accuracy on test set: {accuracy:.4f}")

test()

In [ ]:

model_resnet_residuals.view_graph(input_data=test_images[:1],filename="model_graph", format="png", view=True)

In [ ]:
model_resnet_residuals.save_model(filepath="model_resnet.h5")